In [180]:
import numpy as np
import pandas as pd
import sys
import time
import math

In [263]:
epsilon = 10**-8
n=785
M=100

In [182]:
train_df = pd.read_csv("./Alphabets/train.csv")
train_data = train_df.values

test_df = pd.read_csv("./Alphabets/test.csv")
test_data = test_df.values

In [202]:
#=========================READING TRAINING DATA================================

# X_train = np.zeros((np.shape(train_data)[0], np.shape(train_data)[1]), dtype=float)
X_train = train_data[:,:-1]
Y_data = train_data[:,-1]
Y_train = np.zeros((np.shape(X_train)[0],26))

#converting input into one-hot encoding
for i in range(np.shape(Y_data)[0]):
    Y_train[i][Y_data[i]] = 1  

X_train = np.insert(X_train,0,1.,axis=1)
    
X_train = X_train/255.

print(X_train.shape)
#========================READING TEST DATA======================================

X_test = test_data[:,:-1]
Y_data = test_data[:,-1]
Y_test = np.zeros((np.shape(X_test)[0],26))

#converting input into one-hot encoding
for i in range(np.shape(Y_data)[0]):
    Y_test[i][Y_data[i]] = 1  
    
X_test = np.insert(X_test,0,1.,axis=1)
    
X_test = X_test/255.

print(X_test.shape)

(12999, 785)
(6499, 785)


In [203]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def sig_prime(x):
    return (sigmoid(x))*(1-sigmoid(x))

def ReLU(x):
    return np.maximum(0,x)

def ReLU_prime(x):
    return np.where(x>0, 1, 0)

def activation(type,input):
    if (type=='sig'):
        return sigmoid(input)
    elif(type=='relu'):
        return ReLU(input)
    
print(ReLU_prime(np.asarray([[2.4332,34,23234,-0.324,0],[23.45,-6.78,5.67,6.8,0]])))

[[1 1 1 0 0]
 [1 0 1 1 0]]


In [204]:
def one_hot(output):
    ans = np.zeros((26))
    ans[np.argmax(output)] = 1
    return ans

def mse(y_true,y_pred):
    return np.sum(np.power(y_true-y_pred,2))

def mse_batch(true_batch,pred_batch,M,b):
    error=0

    for i in range((b-1)*M,b*M):
        error+=mse(true_batch[i],pred_batch[i-((b-1)*M)])    
    return (error/(2*M))

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

def mse_batch_prime(true_batch,pred_batch,M,input_size,output_size,input):
    total_error = np.zeros((output_size,input_size))
    for i in range(M):
        output_error = mse_prime(true_batch[i], pred_batch[i])
        total_error += np.dot(input[i].T, output_error)
        
    return (total_error) 

In [205]:
def init_theta(Theta,layer):
    Theta[0] = np.random.randn(layers[0], 785) - 0.5
    Theta[0] = Theta[0]/5
    for i in range(1, len(layers)):
        Theta[i] = np.random.randn(layers[i], layers[i-1]+1) - 0.5
        Theta[i] = Theta[i]/5

In [271]:
layers = []
layers.append(100)
layers.append(100)
layers.append(26)

Theta = {}
init_theta(Theta,layers)

In [272]:
print(layers)
print(Theta[0].shape)
print(Theta[1].shape)

[100, 100, 26]
(100, 785)
(100, 101)


In [273]:
def forward_prop(Theta,X,type):
    Output = {}
    out = np.dot(X, Theta[0].T)
    out = activation(type,out)
    Output[0] = np.insert(out,0,1,axis=1)
    
    for i in range(1,len(layers)-1):
        out = np.dot(Output[i-1], Theta[i].T)
        out = activation(type,out)
        out = np.insert(out,0,1.,axis=1)   
        Output[i] = out

    out = np.dot(Output[len(layers)-2], Theta[len(layers)-1].T)
    out = activation('sig',out)
    Output[len(layers)-1] = out
    
    return Output         

In [274]:
def update_weight(Theta,X,Y,learning_rate,activation,is_deep):
    output = forward_prop(Theta,X,activation)
    count = len(layers)+1
    layer_out = output[len(layers)-1]
    layer_in = output[len(layers)-2]
    curr_delta = np.multiply(np.multiply((Y-layer_out), layer_out), (1-layer_out))
    if (is_deep==True):
        Delta = {}
        Delta[len(layers)-1] = curr_delta
    gradient = -np.matmul(curr_delta.T,layer_in)/M
    Theta[len(layers)-1] = Theta[len(layers)-1] - (learning_rate*gradient)
    count -= 1
    
    for i in range(len(layers)-2,0,-1):
        layer_out = output[i]
        layer_in = output[i-1]
        curr_delta = np.dot(Delta[i+1],Theta[i+1])
        if (activation=='sig'):
            curr_delta = np.multiply(np.multiply((curr_delta), layer_out), (1-layer_out))
        elif (activation=='relu'):
            curr_delta = np.multiply(curr_delta,ReLU_prime(layer_out))
        curr_delta = curr_delta[:, 1:]
        if (is_deep==True):
            Delta[i] = curr_delta
        gradient = -np.matmul(curr_delta.T, layer_in)/M
        Theta[i] = Theta[i] - (learning_rate*gradient)
        count -= 1

    layer_out = output[0]
    layer_in = X
    curr_delta = np.dot(Delta[1],Theta[1])
    if (activation=='sig'):
        curr_delta = np.multiply(np.multiply(curr_delta, layer_out), (1-layer_out))
    elif (activation=='relu'):
        curr_delta = np.multiply(curr_delta,ReLU_prime(layer_out))
    curr_delta = curr_delta[:, 1:]
    if (is_deep==True):
        Delta[0] = curr_delta
    gradient = -np.matmul(curr_delta.T, layer_in)/M
    Theta[0] = Theta[0] - (learning_rate*gradient)    
    count -= 1

In [275]:
def mse_error(Y_True,Y_pred):
    return np.sum(np.power(Y_True-Y_pred,2))/(2*M)

def adaptive_rate(init_rate,epoch_count):
    return float(init_rate/(epoch_count**(1/2)))
                 
def toggle(to_do,init_rate,epoch_count):
    if (to_do==True):
        return adaptive_rate(init_rate,epoch_count)
    else:
        return init_rate

In [284]:
prev_err = 550
curr_err = 500
init_rate = 0.5
epochs = 0
to_do = False
is_deep = True


start = time.time()
while(abs(prev_err-curr_err)>epsilon and epochs<7000):
    prev_err = curr_err
    curr_err = 0
    epochs+=1
    learning_rate = toggle(to_do,init_rate,epochs) 
    
    for b in range(1,int(len(X_train)/100)):
        
        update_weight(Theta,X_train[((b-1)*M):(b*M)],Y_train[((b-1)*M):(b*M)],learning_rate,'sig',is_deep)
        
        Y_pred = forward_prop(Theta,X_train[((b-1)*M):(b*M)],'sig')
        
        curr_err += mse_error(Y_train[((b-1)*M):(b*M)],Y_pred[len(layers)-1])
    
    curr = time.time()
    
    print('epoch %d error=%f time=%f'   % (epochs, curr_err,curr-start)) 

epoch 1 error=3.195515 time=0.249047
epoch 2 error=3.195329 time=0.445379
epoch 3 error=3.195081 time=0.674469
epoch 4 error=3.194825 time=0.900240
epoch 5 error=3.194567 time=1.114508
epoch 6 error=3.194307 time=1.319507
epoch 7 error=3.194046 time=1.531692
epoch 8 error=3.193783 time=1.739019
epoch 9 error=3.193519 time=1.953611
epoch 10 error=3.193253 time=2.154959
epoch 11 error=3.192986 time=2.373534
epoch 12 error=3.192716 time=2.572578
epoch 13 error=3.192443 time=2.794166
epoch 14 error=3.192168 time=3.006659
epoch 15 error=3.191888 time=3.225144
epoch 16 error=3.191602 time=3.473017
epoch 17 error=3.191311 time=3.691737
epoch 18 error=3.191011 time=3.902278
epoch 19 error=3.190700 time=4.147339
epoch 20 error=3.190374 time=4.388729
epoch 21 error=3.190029 time=4.614791
epoch 22 error=3.189657 time=4.869211
epoch 23 error=3.189248 time=5.112115
epoch 24 error=3.188788 time=5.348128
epoch 25 error=3.188261 time=5.579877
epoch 26 error=3.187654 time=5.917601
epoch 27 error=3.1869

epoch 210 error=3.129399 time=52.359567
epoch 211 error=3.128787 time=52.613545
epoch 212 error=3.128224 time=52.865182
epoch 213 error=3.127733 time=53.112661
epoch 214 error=3.127297 time=53.357091
epoch 215 error=3.126878 time=53.602767
epoch 216 error=3.126449 time=53.847831
epoch 217 error=3.125991 time=54.092698
epoch 218 error=3.125495 time=54.338237
epoch 219 error=3.124964 time=54.583218
epoch 220 error=3.124418 time=54.827359
epoch 221 error=3.123898 time=55.072419
epoch 222 error=3.123444 time=55.315842
epoch 223 error=3.123069 time=55.560358
epoch 224 error=3.122756 time=55.812227
epoch 225 error=3.122480 time=56.058623
epoch 226 error=3.122222 time=56.302905
epoch 227 error=3.121974 time=56.548469
epoch 228 error=3.121730 time=56.794506
epoch 229 error=3.121490 time=57.039426
epoch 230 error=3.121251 time=57.286428
epoch 231 error=3.121015 time=57.530659
epoch 232 error=3.120779 time=57.776637
epoch 233 error=3.120543 time=58.023105
epoch 234 error=3.120307 time=58.268149


epoch 415 error=3.056461 time=105.253886
epoch 416 error=3.056124 time=105.520317
epoch 417 error=3.055781 time=105.785160
epoch 418 error=3.055437 time=106.050521
epoch 419 error=3.055097 time=106.313556
epoch 420 error=3.054767 time=106.578639
epoch 421 error=3.054453 time=106.844124
epoch 422 error=3.054156 time=107.109699
epoch 423 error=3.053877 time=107.377742
epoch 424 error=3.053615 time=107.650980
epoch 425 error=3.053368 time=107.922731
epoch 426 error=3.053134 time=108.196412
epoch 427 error=3.052910 time=108.466769
epoch 428 error=3.052695 time=108.739681
epoch 429 error=3.052487 time=109.010041
epoch 430 error=3.052284 time=109.281078
epoch 431 error=3.052087 time=109.554200
epoch 432 error=3.051892 time=109.824320
epoch 433 error=3.051701 time=110.089238
epoch 434 error=3.051512 time=110.354370
epoch 435 error=3.051324 time=110.619164
epoch 436 error=3.051137 time=110.884553
epoch 437 error=3.050951 time=111.158599
epoch 438 error=3.050764 time=111.420898
epoch 439 error=

epoch 615 error=3.004665 time=157.267590
epoch 616 error=3.004548 time=157.523577
epoch 617 error=3.004431 time=157.779534
epoch 618 error=3.004314 time=158.035567
epoch 619 error=3.004197 time=158.291375
epoch 620 error=3.004081 time=158.547305
epoch 621 error=3.003964 time=158.803349
epoch 622 error=3.003847 time=159.059202
epoch 623 error=3.003730 time=159.315124
epoch 624 error=3.003613 time=159.571589
epoch 625 error=3.003496 time=159.831133
epoch 626 error=3.003378 time=160.094102
epoch 627 error=3.003260 time=160.360902
epoch 628 error=3.003141 time=160.625428
epoch 629 error=3.003021 time=160.891623
epoch 630 error=3.002901 time=161.184903
epoch 631 error=3.002779 time=161.455828
epoch 632 error=3.002656 time=161.777384
epoch 633 error=3.002531 time=162.034736
epoch 634 error=3.002404 time=162.344483
epoch 635 error=3.002275 time=162.638052
epoch 636 error=3.002144 time=162.903381
epoch 637 error=3.002009 time=163.163096
epoch 638 error=3.001870 time=163.425109
epoch 639 error=

epoch 815 error=2.957968 time=209.782386
epoch 816 error=2.957814 time=210.046326
epoch 817 error=2.957659 time=210.309867
epoch 818 error=2.957504 time=210.571178
epoch 819 error=2.957348 time=210.834067
epoch 820 error=2.957193 time=211.097917
epoch 821 error=2.957036 time=211.360808
epoch 822 error=2.956880 time=211.625837
epoch 823 error=2.956723 time=211.892091
epoch 824 error=2.956566 time=212.161034
epoch 825 error=2.956409 time=212.432204
epoch 826 error=2.956252 time=212.703540
epoch 827 error=2.956096 time=212.976566
epoch 828 error=2.955940 time=213.240447
epoch 829 error=2.955785 time=213.503396
epoch 830 error=2.955631 time=213.765993
epoch 831 error=2.955479 time=214.028137
epoch 832 error=2.955328 time=214.291586
epoch 833 error=2.955179 time=214.555249
epoch 834 error=2.955032 time=214.820622
epoch 835 error=2.954887 time=215.083035
epoch 836 error=2.954744 time=215.349984
epoch 837 error=2.954604 time=215.612239
epoch 838 error=2.954467 time=215.876771
epoch 839 error=

...

epoch 4908 error=2.604832 time=1252.352608
epoch 4909 error=2.604822 time=1252.595125
epoch 4910 error=2.604813 time=1252.836602
epoch 4911 error=2.604803 time=1253.081988
epoch 4912 error=2.604794 time=1253.321731
epoch 4913 error=2.604784 time=1253.595601
epoch 4914 error=2.604775 time=1253.904710
epoch 4915 error=2.604766 time=1254.176908
epoch 4916 error=2.604756 time=1254.468230
epoch 4917 error=2.604747 time=1254.723145
epoch 4918 error=2.604737 time=1254.979012
epoch 4919 error=2.604728 time=1255.272018
epoch 4920 error=2.604719 time=1255.542561
epoch 4921 error=2.604709 time=1255.810310
epoch 4922 error=2.604700 time=1256.077553
epoch 4923 error=2.604691 time=1256.341160
epoch 4924 error=2.604681 time=1256.599738
epoch 4925 error=2.604672 time=1256.863967
epoch 4926 error=2.604662 time=1257.119288
epoch 4927 error=2.604653 time=1257.372545
epoch 4928 error=2.604644 time=1257.624841
epoch 4929 error=2.604634 time=1257.897110
epoch 4930 error=2.604625 time=1258.165195
epoch 4931 

epoch 5099 error=2.582923 time=1299.664065
epoch 5100 error=2.582739 time=1299.906492
epoch 5101 error=2.582655 time=1300.148659
epoch 5102 error=2.582554 time=1300.392624
epoch 5103 error=2.582452 time=1300.637675
epoch 5104 error=2.582357 time=1300.888386
epoch 5105 error=2.582269 time=1301.127801
epoch 5106 error=2.582185 time=1301.371633
epoch 5107 error=2.582104 time=1301.613225
epoch 5108 error=2.582023 time=1301.853467
epoch 5109 error=2.581940 time=1302.095387
epoch 5110 error=2.581849 time=1302.336116
epoch 5111 error=2.581744 time=1302.580941
epoch 5112 error=2.581614 time=1302.815956
epoch 5113 error=2.581434 time=1303.051081
epoch 5114 error=2.581151 time=1303.284896
epoch 5115 error=2.580639 time=1303.520224
epoch 5116 error=2.579640 time=1303.754195
epoch 5117 error=2.578221 time=1303.988687
epoch 5118 error=2.577653 time=1304.222639
epoch 5119 error=2.577614 time=1304.455454
epoch 5120 error=2.577542 time=1304.692462
epoch 5121 error=2.577450 time=1304.926693
epoch 5122 

epoch 5290 error=2.565793 time=1345.312088
epoch 5291 error=2.565747 time=1345.553732
epoch 5292 error=2.565703 time=1345.792779
epoch 5293 error=2.565661 time=1346.032614
epoch 5294 error=2.565620 time=1346.274648
epoch 5295 error=2.565580 time=1346.514077
epoch 5296 error=2.565541 time=1346.752439
epoch 5297 error=2.565504 time=1346.991017
epoch 5298 error=2.565467 time=1347.231549
epoch 5299 error=2.565431 time=1347.471493
epoch 5300 error=2.565396 time=1347.710394
epoch 5301 error=2.565362 time=1347.948838
epoch 5302 error=2.565328 time=1348.189964
epoch 5303 error=2.565295 time=1348.431424
epoch 5304 error=2.565262 time=1348.671742
epoch 5305 error=2.565230 time=1348.912249
epoch 5306 error=2.565198 time=1349.149961
epoch 5307 error=2.565167 time=1349.390830
epoch 5308 error=2.565136 time=1349.629202
epoch 5309 error=2.565105 time=1349.867110
epoch 5310 error=2.565074 time=1350.105962
epoch 5311 error=2.565044 time=1350.345260
epoch 5312 error=2.565013 time=1350.587371
epoch 5313 

epoch 5481 error=2.551329 time=1392.042175
epoch 5482 error=2.551306 time=1392.283034
epoch 5483 error=2.551283 time=1392.523548
epoch 5484 error=2.551260 time=1392.765485
epoch 5485 error=2.551238 time=1393.004945
epoch 5486 error=2.551215 time=1393.242816
epoch 5487 error=2.551192 time=1393.481327
epoch 5488 error=2.551170 time=1393.723463
epoch 5489 error=2.551147 time=1393.964339
epoch 5490 error=2.551125 time=1394.202491
epoch 5491 error=2.551103 time=1394.442905
epoch 5492 error=2.551081 time=1394.682352
epoch 5493 error=2.551058 time=1394.921715
epoch 5494 error=2.551036 time=1395.159792
epoch 5495 error=2.551014 time=1395.396735
epoch 5496 error=2.550992 time=1395.637799
epoch 5497 error=2.550970 time=1395.876461
epoch 5498 error=2.550947 time=1396.116432
epoch 5499 error=2.550925 time=1396.355726
epoch 5500 error=2.550903 time=1396.596759
epoch 5501 error=2.550880 time=1396.835436
epoch 5502 error=2.550858 time=1397.074009
epoch 5503 error=2.550835 time=1397.315665
epoch 5504 

KeyboardInterrupt: 

In [285]:
print(learning_rate)

0.5


In [286]:
count = 0
output = forward_prop(Theta,X_train,'sig')
for i in range(len(Y_train)):
    if (np.argmax(Y_train[i])==np.argmax(output[len(layers)-1][i])):
        count+=1
        
print("Training accuracy is",(count/len(Y_train))*100,"%")

Training accuracy is 96.06892837910608 %


In [287]:
count = 0
output = forward_prop(Theta,X_test,'sig')
for i in range(len(Y_test)):
    if (np.argmax(Y_test[i])==np.argmax(output[len(layers)-1][i])):
        count+=1
        
print("Test accuracy is",(count/len(Y_test))*100,"%")

Test accuracy is 84.2129558393599 %
